### 결측치 처리 
---------
#### 1. 일반적인 처리 방법
#### 2. sklearn을 이용한 처리 방법
#### 3. pandas를 이용한 처리 방법

In [1]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = {
    'id': [1000, 1001, 1002, 1003, 1004, 1005, 1006],
    'date': ['20230101','20230102','20230103','20230104','20230105','20230106','20230107'],
    'age': [21, 56, 33, 48, 27, 42, 32],
    'income': [67000, 220000, 97000, 166000, 81000, 157000, 96000],
    'gender': ['Male', 'Female', 'Female', 'Male', 'Male', 'Female', 'Female'],
    'education': ['Bachelors', 'PhD', 'Masters', 'Masters', 'Bachelors', 'Bachelors', 'Bachelors'], 
    'passed': [False, True, True, True, False, False, True],
    'measurement': np.random.randn(7).round(2)
}

df = pd.DataFrame(data)
df

,id,date,age,income,gender,education,passed,measurement
0,1000,20230101,21,67000,Male,Bachelors,False,0.42
1,1001,20230102,56,220000,Female,PhD,True,-0.43
2,1002,20230103,33,97000,Female,Masters,True,-0.31
3,1003,20230104,48,166000,Male,Masters,True,-1.21
4,1004,20230105,27,81000,Male,Bachelors,False,1.82
5,1005,20230106,42,157000,Female,Bachelors,False,1.08
6,1006,20230107,32,96000,Female,Bachelors,True,-0.93


In [3]:
# 컬럼간 상관계수 계산하기

In [4]:
df.corr()

,id,age,income,passed,measurement
id,1.000000,-0.006285,-0.076124,0.000000,0.077271
age,-0.006285,1.000000,0.984332,0.533450,-0.434061
income,-0.076124,0.984332,1.000000,0.413131,-0.340988
passed,0.000000,0.533450,0.413131,1.000000,-0.889007
measurement,0.077271,-0.434061,-0.340988,-0.889007,1.000000


In [5]:
# 컬럼 정보 확인하기

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           7 non-null      int64  
 1   date         7 non-null      object 
 2   age          7 non-null      int64  
 3   income       7 non-null      int64  
 4   gender       7 non-null      object 
 5   education    7 non-null      object 
 6   passed       7 non-null      bool   
 7   measurement  7 non-null      float64
dtypes: bool(1), float64(1), int64(3), object(3)
memory usage: 527.0+ bytes


In [7]:
# 데이터 값 살펴보기

In [8]:
df.values

array([[1000, '20230101', 21, 67000, 'Male', 'Bachelors', False, 0.42],
       [1001, '20230102', 56, 220000, 'Female', 'PhD', True, -0.43],
       [1002, '20230103', 33, 97000, 'Female', 'Masters', True, -0.31],
       [1003, '20230104', 48, 166000, 'Male', 'Masters', True, -1.21],
       [1004, '20230105', 27, 81000, 'Male', 'Bachelors', False, 1.82],
       [1005, '20230106', 42, 157000, 'Female', 'Bachelors', False, 1.08],
       [1006, '20230107', 32, 96000, 'Female', 'Bachelors', True, -0.93]],
      dtype=object)

In [9]:
# 임의로 결측치 삽입하기

In [10]:
df2 = df.copy()
df2.iloc[[3, 5], [1, 2, 4, 5, 6, 7]] = np.nan
df2

,id,date,age,income,gender,education,passed,measurement
0,1000,20230101,21.0,67000,Male,Bachelors,False,0.42
1,1001,20230102,56.0,220000,Female,PhD,True,-0.43
2,1002,20230103,33.0,97000,Female,Masters,True,-0.31
3,1003,NaN,NaN,166000,NaN,NaN,NaN,NaN
4,1004,20230105,27.0,81000,Male,Bachelors,False,1.82
5,1005,NaN,NaN,157000,NaN,NaN,NaN,NaN
6,1006,20230107,32.0,96000,Female,Bachelors,True,-0.93


In [11]:
#각 컬럼별 결측치 세어보기

In [12]:
df2.isnull().sum()

id             0
date           2
age            2
income         0
gender         2
education      2
passed         2
measurement    2
dtype: int64

In [13]:
# 결측이 포함된 데이터만 보기

In [14]:
df2[df2.isnull().any(axis=1)]

,id,date,age,income,gender,education,passed,measurement
3,1003,NaN,NaN,166000,NaN,NaN,NaN,NaN
5,1005,NaN,NaN,157000,NaN,NaN,NaN,NaN


In [15]:
# 결측치가 포함된 행 삭제

In [16]:
df2.dropna(axis=0) 

,id,date,age,income,gender,education,passed,measurement
0,1000,20230101,21.0,67000,Male,Bachelors,False,0.42
1,1001,20230102,56.0,220000,Female,PhD,True,-0.43
2,1002,20230103,33.0,97000,Female,Masters,True,-0.31
4,1004,20230105,27.0,81000,Male,Bachelors,False,1.82
6,1006,20230107,32.0,96000,Female,Bachelors,True,-0.93


In [17]:
# 결측치가 포함된 열 삭제

In [18]:
df2.dropna(axis=1) #결측치가 포함된 column drop

,id,income
0,1000,67000
1,1001,220000
2,1002,97000
3,1003,166000
4,1004,81000
5,1005,157000
6,1006,96000


In [19]:
# 결측치를 특정 값으로 치환하기

In [20]:
df2.loc[:, ['age', 'measurement']].fillna(-9999)

,age,measurement
0,21.0,0.42
1,56.0,-0.43
2,33.0,-0.31
3,-9999.0,-9999.00
4,27.0,1.82
5,-9999.0,-9999.00
6,32.0,-0.93


In [21]:
# 결측치를 열 평균값으로 치환하기

In [22]:
df2.loc[:, ['age', 'measurement']].fillna(df2.mean()[['age','measurement']])

,age,measurement
0,21.0,0.420
1,56.0,-0.430
2,33.0,-0.310
3,33.8,0.114
4,27.0,1.820
5,33.8,0.114
6,32.0,-0.930


In [ ]:
# sklearn 패키지를 활용하여 결측치 처리하기
# Imputer를 활용하여 결측치 채워넣기 (특정값)

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-9999)
imputed = imp.fit_transform(df2.loc[:, ['age', 'measurement']].values)
pd.DataFrame(imputed, columns=['age', 'measurement'])

,age,measurement
0,21.0,-1.03
1,56.0,-1.10
2,33.0,1.31
3,-9999.0,-9999.00
4,27.0,0.22
5,-9999.0,-9999.00
6,32.0,-0.65


In [23]:
# Imputer를 활용하여 결측치 채워넣기 (평균)

In [24]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed = imp.fit_transform(df2.loc[:, ['age', 'measurement']].values)
pd.DataFrame(imputed, columns=['age', 'measurement'])

,age,measurement
0,21.0,0.420
1,56.0,-0.430
2,33.0,-0.310
3,33.8,0.114
4,27.0,1.820
5,33.8,0.114
6,32.0,-0.930


In [25]:
# Imputer를 활용하여 결측치 채워넣기 (n_neighbor)

In [27]:
knn_imp = KNNImputer(n_neighbors=2, weights="uniform")
imputed = knn_imp.fit_transform(df2.loc[:, ['age', 'measurement']].values)
pd.DataFrame(imputed, columns=['age', 'measurement'])

,age,measurement
0,21.0,0.420
1,56.0,-0.430
2,33.0,-0.310
3,33.8,0.114
4,27.0,1.820
5,33.8,0.114
6,32.0,-0.930


In [ ]:
# 문자열에 대한 결측치 처리하기
# 특정값으로 치환하기

In [28]:
df2.loc[:, ['gender', 'education']].fillna('Unknown') 

,gender,education
0,Male,Bachelors
1,Female,PhD
2,Female,Masters
3,Unknown,Unknown
4,Male,Bachelors
5,Unknown,Unknown
6,Female,Bachelors


In [29]:
# mode(최빈값)을 활용하여 치환하기

In [30]:
df2.loc[:, ['gender', 'education']].fillna(df2.mode().iloc[0]) #최빈값 함수 mode를 활용하여 채우기

,gender,education
0,Male,Bachelors
1,Female,PhD
2,Female,Masters
3,Female,Bachelors
4,Male,Bachelors
5,Female,Bachelors
6,Female,Bachelors


In [ ]:
# sklearn을 활용하여 문자열 결측치 처리하기

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='Unknown')
imputed = imp.fit_transform(df2.loc[:, ['gender', 'education']].values)
pd.DataFrame(imputed, columns=['gender', 'education'])

,gender,education
0,Male,Bachelors
1,Female,PhD
2,Female,Masters
3,Unknown,Unknown
4,Male,Bachelors
5,Unknown,Unknown
6,Female,Bachelors


In [31]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputed = imp.fit_transform(df2.loc[:, ['gender', 'education']].values)
pd.DataFrame(imputed, columns=['gender', 'education'])

,gender,education
0,Male,Bachelors
1,Female,PhD
2,Female,Masters
3,Female,Bachelors
4,Male,Bachelors
5,Female,Bachelors
6,Female,Bachelors


In [ ]:
# 문자열 데이터를 날짜 데이터로 처리하기

In [ ]:
df2.date

0    20230101
1    20230102
2    20230103
3         NaN
4    20230105
5         NaN
6    20230107
Name: date, dtype: object

In [ ]:
df2.date.dtype

dtype('O')

In [ ]:
df2.date = pd.to_datetime(df2.date) #to_datetime 함수를 활용하여 str -> datetime으로 변환

In [ ]:
df2.date.dtype

dtype('<M8[ns]')

In [ ]:
df2.date[1].dayofweek

0

In [ ]:
df2.date[1].weekofyear

1

In [ ]:
df2['date'] = pd.date_range(start=df2.date.min(), end=df2.date.max(), freq='D')

In [ ]:
df2['dayofweek'] = df2['date'].apply(lambda x: x.dayofweek)

In [ ]:
df2

,id,date,age,income,gender,education,passed,measurement,dayofweek
0,1000,2023-01-01,21.0,67000,Male,Bachelors,False,-1.03,6
1,1001,2023-01-02,56.0,220000,Female,PhD,True,-1.10,0
2,1002,2023-01-03,33.0,97000,Female,Masters,True,1.31,1
3,1003,2023-01-04,NaN,166000,NaN,NaN,NaN,NaN,2
4,1004,2023-01-05,27.0,81000,Male,Bachelors,False,0.22,3
5,1005,2023-01-06,NaN,157000,NaN,NaN,NaN,NaN,4
6,1006,2023-01-07,32.0,96000,Female,Bachelors,True,-0.65,5
